In [1]:
    
import PIL
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import matplotlib
matplotlib.use('Agg')
#import tensorflow as tf
import matplotlib.pyplot as plt
from sargan_models import SARGAN
import os
from tqdm import tqdm
from random import shuffle
import skimage.measure as ski_me
import time
import numpy as np
import torch
import json
from cifar_helper import get_data, chunks
from sar_utilities import add_gaussian_noise, preprocess_test
from alert_utilities import send_images_via_email

from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
import torchvision
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from scipy.ndimage.filters import gaussian_filter
from pgd_attack import LinfPGDAttack
from model import Model
#import tensorflow_datasets as tfds
with open('pgd/config.json') as config_file:
    config = json.load(config_file)


C:\Users\Hans Walker\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Hans Walker\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Hans Walker\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Hans Walker\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:5

Instructions for updating:
non-resource variables are not supported in the long term


C:\Users\Hans Walker\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Hans Walker\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Hans Walker\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Hans Walker\anaconda3\lib\site-packages\tensorboard\compat

In [25]:
img_size = (224,224,1)
trained_model_path = 'trained_models/sargan_mnist_gaussian_corrupted'
BATCH_SIZE = 64
NOISE_STD_RANGE = [0.1, 0.3]

In [97]:
tf.reset_default_graph()
sargan_model=SARGAN(img_size, BATCH_SIZE, img_channel=1)
image=np.load('testimage.NPY')
newimage=np.zeros([28,28])
for i in range(28):
    for j in range(28):
        newimage[i,j]=image[10,i*28+j]
newimage = PIL.Image.fromarray(newimage*255)
transform=Resize((224,224))
transform2=Resize((28,28))
newimage= transform(newimage)
newimage=np.array(newimage).reshape(1,224,224,1)
imagebatch=np.zeros([64,224,224,1])
imagebatch2=np.zeros([64,224,224,1])
imagebatch[0]=newimage
print(imagebatch.shape)
with tf.Session() as sess:
    saver= tf.train.Saver()
    sargan_saver = tf.train.import_meta_graph(trained_model_path+'\sargan_mnist.meta')
    sargan_saver.restore(sess,tf.train.latest_checkpoint(trained_model_path))
    imagebatch2 = gaussian_filter(np.array([add_gaussian_noise(imagebatch, sd=np.random.uniform(NOISE_STD_RANGE[0], NOISE_STD_RANGE[1]*10))]), sigma=2)
    imagebatch2=imagebatch2.reshape((64, 224, 224, 1))
    print(imagebatch2.reshape((64, 224, 224, 1)).shape)
    gen_img = sess.run(sargan_model.gen_img,feed_dict={sargan_model.image: imagebatch, sargan_model.cond: imagebatch2})

(64, 224, 224, 1)
INFO:tensorflow:Restoring parameters from trained_models/sargan_mnist_gaussian_corrupted\sargan_mnist
(64, 224, 224, 1)


In [99]:
image=np.load('testimage.NPY')
print((image.shape))
newbatch=image.reshape([200,28,28,1])
#print(newimage.shape)
newimage = torch.from_numpy(newbatch)
newimage= transform(newimage)
#newimage.show()

(200, 784)


'C:\\Users\\Hans Walker\\Desktop\\Hans the Awesome\\research\\adverserial\\Blur'

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import json
import math
import os
import sys
import time

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

from model import Model
from pgd_attack import LinfPGDAttack


from sargan_models import SARGAN
from sar_utilities import add_gaussian_noise, preprocess_test
from scipy.ndimage.filters import gaussian_filter
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
import torch
import numpy as np
batch=np.load('testimage.NPY')

from PIL import Image
tf.reset_default_graph()







img_size = (224,224,1)
trained_model_path = 'trained_models/sargan_mnist_gaussian_corrupted'
BATCH_SIZE = 64
NOISE_STD_RANGE = [0.1, .9]
    
#sargan model
#reshaping the images to a square
newbatch=batch.reshape([len(batch),28,28])
#upping the size of the image
transform=Compose([ Resize((224, 224)), Normalize((np.mean(batch)), (np.std(batch),))])
#creating the original batch
transformed_batch=np.array(transform(torch.from_numpy(newbatch))).reshape([len(batch),224,224,1])
#corrupting
corruptedbatch=np.zeros([len(transformed_batch),224,224,1])
for i in range(len(transformed_batch)):
    #corruptedbatch[i]=gaussian_filter(np.array([add_gaussian_noise(transformed_batch[i], sd=np.random.uniform(NOISE_STD_RANGE[0], NOISE_STD_RANGE[1]))]), sigma=10)[0]
    corruptedbatch[i]=np.array([add_gaussian_noise(transformed_batch[i], sd=np.random.uniform(NOISE_STD_RANGE[0], NOISE_STD_RANGE[1]))])[0]
print(transformed_batch[0]-corruptedbatch[i])
    #storage spaces for the prcoessed images
cbatch=np.zeros([BATCH_SIZE,224,224,1])
outbatch=np.zeros([len(batch),224,224])
j=0
k=0
tf.reset_default_graph()
sargan_model=SARGAN(img_size, BATCH_SIZE, img_channel=1)
with tf.Session() as sess:
    saver= tf.train.Saver()
    sargan_saver = tf.train.import_meta_graph(trained_model_path+'\sargan_mnist.meta')
    sargan_saver.restore(sess,tf.train.latest_checkpoint(trained_model_path))
    for i in range(len(batch)):
        j+=1
        cbatch[i%BATCH_SIZE]=corruptedbatch[i]
        if((j%BATCH_SIZE==0) or ((i+1)%len(batch)==0)):
            processed_batch=sess.run(sargan_model.gen_img,feed_dict={sargan_model.image: cbatch, sargan_model.cond: cbatch})
            for l in range(j):
                outbatch[k]=processed_batch[l,:,:,0]
                k+=1
            j=0
transform2=Resize([28,28])
outbatch=np.array(transform2(torch.from_numpy(outbatch)))
outbatch=outbatch.reshape([len(batch),784])
testbatch=outbatch
testbatch=testbatch.reshape([len(testbatch),28,28])
#upping the size of the image
transform=Resize([224,224])
#creating the original batch
testbatch=np.array(transform(torch.from_numpy(testbatch))).reshape([len(batch),224,224,1])
print(testbatch[0,:,:,0])
Image.fromarray(testbatch[0,:,:,0]*255).show()

[[[-0.66161723]
  [ 0.63288684]
  [ 0.63479557]
  ...
  [-0.64873736]
  [-0.51696739]
  [-0.08230452]]

 [[ 0.53192286]
  [ 0.37492781]
  [-0.21993874]
  ...
  [ 1.16431846]
  [ 0.27044244]
  [ 1.82052985]]

 [[ 0.99241565]
  [-0.80039343]
  [ 0.5086151 ]
  ...
  [-0.57598887]
  [ 0.01022903]
  [ 0.25040362]]

 ...

 [[-0.84682802]
  [-0.22416307]
  [ 0.50152838]
  ...
  [-0.07882573]
  [-1.71340639]
  [-0.12079074]]

 [[-1.1001386 ]
  [ 0.55443364]
  [-0.35943609]
  ...
  [ 0.46351164]
  [ 0.72860994]
  [ 0.32627173]]

 [[-0.6635486 ]
  [-0.79256161]
  [ 0.27732326]
  ...
  [-0.45064955]
  [ 0.04858221]
  [ 0.64657731]]]
INFO:tensorflow:Restoring parameters from trained_models/sargan_mnist_gaussian_corrupted\sargan_mnist
[[-0.328371   -0.328371   -0.328371   ... -0.3957676  -0.3957676
  -0.3957676 ]
 [-0.328371   -0.328371   -0.328371   ... -0.3957676  -0.3957676
  -0.3957676 ]
 [-0.328371   -0.328371   -0.328371   ... -0.3957676  -0.3957676
  -0.3957676 ]
 ...
 [-0.37163928 -0.371639

In [6]:
batch2=np.load('original_images.NPY')







img_size = (224,224,1)
trained_model_path = 'trained_models/sargan_mnist_gaussian_corrupted'
BATCH_SIZE = 64
NOISE_STD_RANGE = [0.1, .9]
    
#sargan model
#reshaping the images to a square
newbatch2=batch.reshape([len(batch2),28,28])
#upping the size of the image
transform2=Compose([ Resize((224, 224)), Normalize((np.mean(batch2)), (np.std(batch2),))])
#creating the original batch
transformed_batch2=np.array(transform(torch.from_numpy(newbatch2))).reshape([len(batch2),224,224,1])

In [8]:
testbatch=outbatch
testbatch=testbatch.reshape([len(testbatch),28,28])
#upping the size of the image
transform=Resize([224,224])
#creating the original batch
testbatch=np.array(transform(torch.from_numpy(testbatch))).reshape([len(batch),224,224,1])
Image.fromarray(transformed_batch2[0,:,:,0]*255).show()
#Image.fromarray(corruptedbatch[0,:,:,0]*255).show()
Image.fromarray(transformed_batch[0,:,:,0]*255).show()

In [4]:
#downgrading the transform
outbatch2=outbatch
#ncbatch=corruptedbatch
print(type(outbatch2))
#Image.fromarray(ncbatch[0,:,:,0]*255).show()
#Image.fromarray(outbatch2[0]*255).show()
transform2=Resize([28,28])
Image.fromarray(outbatch2[0]*255).show()
outbatch2=np.array(transform2(torch.from_numpy(outbatch2)))
outbatch2=outbatch2.reshape([len(batch),784])

<class 'numpy.ndarray'>


In [19]:
testbatch=outbatch
testbatch=testbatch.reshape([len(testbatch),28,28])
#upping the size of the image
transform=Resize([224,224])
#creating the original batch
testbatch=np.array(transform(torch.from_numpy(testbatch))).reshape([len(batch),224,224,1])
Image.fromarray(testbatch[10,:,:,0]*-255).show()

In [22]:
image=np.ones([200,200])*200
Image.fromarray(image).show()